Connected to kaggle_llm (Python 3.11.8)

In [1]:
import glob
import os
from datetime import datetime, timedelta

# Get the current time
now = datetime.now()

# Calculate the time 24 hours ago from the current time
twenty_four_hours_ago = now  # - timedelta(hours=24)

# Pattern to match the files with varying numbers in the file path
file_pattern1 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemma_7b_it_rewrites_*.json"
file_pattern2 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_*.json"

# Use glob to find all file paths that match the patterns
file_paths1 = glob.glob(file_pattern1)
file_paths2 = glob.glob(file_pattern2)
file_paths = file_paths1 + file_paths2

# Filter files that were modified within the last 24 hours
recent_file_paths = [
    file
    for file in file_paths
    if datetime.fromtimestamp(os.path.getmtime(file)) < twenty_four_hours_ago
]

# Output the list of recent file paths
print("Files not modified in the last 24 hours:")
for file_path in recent_file_paths:
    print(file_path)

Files not modified in the last 24 hours:
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344663.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344954.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344955.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344956.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344957.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344958.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344967.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344976.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/cha

In [2]:
import json
from datasets import Dataset
import re

# Initialize an empty list to store the extracted entries
entries = []

# Iterate over the recent file paths
for file_path in recent_file_paths:
    # Open the JSON file and load its contents
    with open(file_path, "r") as file:
        file_contents = file.read()
        entries_raw = re.findall(r"\{.*?\}", file_contents, re.DOTALL)
        for entry_raw in entries_raw:
            try:
                entry = json.loads(entry_raw)

                if isinstance(entry["rewrite_prompt"], float):
                    continue

                patterns = [
                    r"^.*?[hH]ere is.*?\n\n",
                    r"^.*?[hH]ere \'s.*?\n\n",
                    r"^.*?[Ss]ure, here*?\n\n",
                ]

                filtered = entry["rewritten_text"]
                for pattern in patterns:
                    filtered = re.sub(pattern, "", filtered, flags=re.DOTALL)

                extracted_entry = {
                    "rewrite_prompt": entry["rewrite_prompt"],
                    "original_text": entry["original_text"],
                    # "rewritten_text": entry["rewritten_text"],
                    "rewritten_text": str(filtered),
                }
                entries.append(extracted_entry)

            except json.JSONDecodeError:
                continue

In [3]:
# Create a HuggingFace Dataset from the extracted entries
dataset = Dataset.from_list(entries)

In [4]:
from datasets import DatasetDict
import random

# Set the random seed for reproducibility
random.seed(42)

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.2)

# Create a DatasetDict from the split datasets
dataset_dict = DatasetDict(
    {"train": train_test_split["train"], "test": train_test_split["test"]}
)

# Save the dataset to a file
dataset_dict.save_to_disk(
    "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemini_categorized_prompts/dataset_train_test_split.hf"
)

Saving the dataset (0/1 shards):   0%|          | 0/39114 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9779 [00:00<?, ? examples/s]

In [5]:
import glob
import os
from datetime import datetime, timedelta

# Get the current time
now = datetime.now()

# Calculate the time 24 hours ago from the current time
twenty_four_hours_ago = now  # - timedelta(hours=24)

# Pattern to match the files with varying numbers in the file path
file_pattern1 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemma_7b_it_rewrites_*.json"
file_pattern2 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_*.json"

# Use glob to find all file paths that match the patterns
file_paths1 = glob.glob(file_pattern1)
file_paths2 = glob.glob(file_pattern2)
file_paths = file_paths1 + file_paths2

# Filter files that were modified within the last 24 hours
# recent_file_paths = [
#     file
#     for file in file_paths
#     if datetime.fromtimestamp(os.path.getmtime(file)) < twenty_four_hours_ago
# ]

# Output the list of recent file paths
print("Files not modified in the last 24 hours:")
for file_path in file_paths:
    print(file_path)

Files not modified in the last 24 hours:
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344663.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344954.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344955.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344956.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344957.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344958.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344967.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344976.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/cha

In [6]:
import json
from datasets import Dataset
import re

# Initialize an empty list to store the extracted entries
entries = []

# Iterate over the recent file paths
for file_path in recent_file_paths:
    # Open the JSON file and load its contents
    with open(file_path, "r") as file:
        file_contents = file.read()
        entries_raw = re.findall(r"\{.*?\}", file_contents, re.DOTALL)
        for entry_raw in entries_raw:
            try:
                entry = json.loads(entry_raw)

                if isinstance(entry["rewrite_prompt"], float):
                    continue

                patterns = [
                    r"^.*?[hH]ere is.*?\n\n",
                    r"^.*?[hH]ere \'s.*?\n\n",
                    r"^.*?[Ss]ure, here*?\n\n",
                ]

                filtered = entry["rewritten_text"]
                for pattern in patterns:
                    filtered = re.sub(pattern, "", filtered, flags=re.DOTALL)

                extracted_entry = {
                    "rewrite_prompt": entry["rewrite_prompt"],
                    "original_text": entry["original_text"],
                    # "rewritten_text": entry["rewritten_text"],
                    "rewritten_text": str(filtered),
                }
                entries.append(extracted_entry)

            except json.JSONDecodeError:
                continue

In [7]:
# Create a HuggingFace Dataset from the extracted entries
dataset = Dataset.from_list(entries)

In [8]:
from datasets import DatasetDict
import random

# Set the random seed for reproducibility
random.seed(42)

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.2)

# Create a DatasetDict from the split datasets
dataset_dict = DatasetDict(
    {"train": train_test_split["train"], "test": train_test_split["test"]}
)

# Save the dataset to a file
dataset_dict.save_to_disk(
    "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemini_categorized_prompts/dataset_train_test_split.hf"
)

Saving the dataset (0/1 shards):   0%|          | 0/39114 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9779 [00:00<?, ? examples/s]

In [9]:
import glob
import os
from datetime import datetime, timedelta

# Get the current time
now = datetime.now()

# Calculate the time 24 hours ago from the current time
twenty_four_hours_ago = now  # - timedelta(hours=24)

# Pattern to match the files with varying numbers in the file path
file_pattern1 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemma_7b_it_rewrites_*.json"
file_pattern2 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_*.json"

# Use glob to find all file paths that match the patterns
file_paths1 = glob.glob(file_pattern1)
file_paths2 = glob.glob(file_pattern2)
file_paths = file_paths1 + file_paths2

# Filter files that were modified within the last 24 hours
# recent_file_paths = [
#     file
#     for file in file_paths
#     if datetime.fromtimestamp(os.path.getmtime(file)) < twenty_four_hours_ago
# ]

# Output the list of recent file paths
print("Files not modified in the last 24 hours:")
for file_path in file_paths:
    print(file_path)

Files not modified in the last 24 hours:
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344663.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344954.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344955.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344956.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344957.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344958.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344967.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344976.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/cha

In [10]:
import json
from datasets import Dataset
import re

# Initialize an empty list to store the extracted entries
entries = []

# Iterate over the recent file paths

In [11]:
import pandas as pd

df = pd.read_csv(
    "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemini_categorized_prompts/prompts.csv"
).set_index("Prompt")

for file_path in file_paths:
    # Open the JSON file and load its contents
    with open(file_path, "r") as file:
        file_contents = file.read()
        entries_raw = re.findall(r"\{.*?\}", file_contents, re.DOTALL)
        for entry_raw in entries_raw:
            try:
                entry = json.loads(entry_raw)

                if isinstance(entry["rewrite_prompt"], float):
                    continue

                patterns = [
                    r"^.*?[hH]ere is.*?\n\n",
                    r"^.*?[hH]ere \'s.*?\n\n",
                    r"^.*?[Ss]ure, here*?\n\n",
                ]

                filtered = entry["rewritten_text"]
                for pattern in patterns:
                    filtered = re.sub(pattern, "", filtered, flags=re.DOTALL)

                extracted_entry = {
                    "rewrite_prompt": entry["rewrite_prompt"],
                    "original_text": entry["original_text"],
                    "rewritten_text": str(filtered),
                    "category": df.loc[entry["rewrite_prompt"], "Category"],
                }
                entries.append(extracted_entry)

            except json.JSONDecodeError:
                continue

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 2


In [ ]:
# Create a HuggingFace Dataset from the extracted entries
dataset = Dataset.from_list(entries)

: 

In [ ]:
from datasets import DatasetDict
import random

# Set the random seed for reproducibility
random.seed(42)

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.2)

# Create a DatasetDict from the split datasets
dataset_dict = DatasetDict(
    {"train": train_test_split["train"], "test": train_test_split["test"]}
)

# Save the dataset to a file
dataset_dict.save_to_disk(
    "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemini_categorized_prompts/dataset_train_test_split.hf"
)

: 

In [12]:
import glob
import os
from datetime import datetime, timedelta

# Get the current time
now = datetime.now()

# Calculate the time 24 hours ago from the current time
twenty_four_hours_ago = now  # - timedelta(hours=24)

# Pattern to match the files with varying numbers in the file path
file_pattern1 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemma_7b_it_rewrites_*.json"
file_pattern2 = "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_*.json"

# Use glob to find all file paths that match the patterns
file_paths1 = glob.glob(file_pattern1)
file_paths2 = glob.glob(file_pattern2)
file_paths = file_paths1 + file_paths2

# Filter files that were modified within the last 24 hours
# recent_file_paths = [
#     file
#     for file in file_paths
#     if datetime.fromtimestamp(os.path.getmtime(file)) < twenty_four_hours_ago
# ]

# Output the list of recent file paths
print("Files not modified in the last 24 hours:")
for file_path in file_paths:
    print(file_path)

Files not modified in the last 24 hours:
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344663.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344954.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344955.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344956.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344957.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344958.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344967.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/chatgpt/gemma_7b_it_rewrites_41344976.json
/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/cha

In [13]:
import json
from datasets import Dataset
import re

# Initialize an empty list to store the extracted entries
entries = []

# Iterate over the recent file paths

In [14]:
import pandas as pd

df = pd.read_csv(
    "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemini_categorized_prompts/prompts.csv",
    delimitter=";",
).set_index("Prompt")

for file_path in file_paths:
    # Open the JSON file and load its contents
    with open(file_path, "r") as file:
        file_contents = file.read()
        entries_raw = re.findall(r"\{.*?\}", file_contents, re.DOTALL)
        for entry_raw in entries_raw:
            try:
                entry = json.loads(entry_raw)

                if isinstance(entry["rewrite_prompt"], float):
                    continue

                patterns = [
                    r"^.*?[hH]ere is.*?\n\n",
                    r"^.*?[hH]ere \'s.*?\n\n",
                    r"^.*?[Ss]ure, here*?\n\n",
                ]

                filtered = entry["rewritten_text"]
                for pattern in patterns:
                    filtered = re.sub(pattern, "", filtered, flags=re.DOTALL)

                extracted_entry = {
                    "rewrite_prompt": entry["rewrite_prompt"],
                    "original_text": entry["original_text"],
                    "rewritten_text": str(filtered),
                    "category": df.loc[entry["rewrite_prompt"], "Category"],
                }
                entries.append(extracted_entry)

            except json.JSONDecodeError:
                continue

TypeError: read_csv() got an unexpected keyword argument 'delimitter'

In [15]:
import pandas as pd

df = pd.read_csv(
    "/scratch/yousefi.m/projects/kaggle_llm_prompt_recovery/data/interim/gemini_categorized_prompts/prompts.csv",
    delimiter=";",
).set_index("Prompt")

for file_path in file_paths:
    # Open the JSON file and load its contents
    with open(file_path, "r") as file:
        file_contents = file.read()
        entries_raw = re.findall(r"\{.*?\}", file_contents, re.DOTALL)
        for entry_raw in entries_raw:
            try:
                entry = json.loads(entry_raw)

                if isinstance(entry["rewrite_prompt"], float):
                    continue

                patterns = [
                    r"^.*?[hH]ere is.*?\n\n",
                    r"^.*?[hH]ere \'s.*?\n\n",
                    r"^.*?[Ss]ure, here*?\n\n",
                ]

                filtered = entry["rewritten_text"]
                for pattern in patterns:
                    filtered = re.sub(pattern, "", filtered, flags=re.DOTALL)

                extracted_entry = {
                    "rewrite_prompt": entry["rewrite_prompt"],
                    "original_text": entry["original_text"],
                    "rewritten_text": str(filtered),
                    "category": df.loc[entry["rewrite_prompt"], "Category"],
                }
                entries.append(extracted_entry)

            except json.JSONDecodeError:
                continue

KeyError: 'Infuse the text with elements of magical realism or fabulism, blurring the boundaries between reality and fantasy to create a sense of wonder and enchantment.'

In [16]:
df

,Category
Prompt,
Condense this into a single tweet.,Brevity
Reduce this text to a maximum of 50 words.,Brevity
Rewrite this in the style of a haiku.,Brevity
Summarize this text in a 10-word elevator pitch.,Brevity
"Shorten this to its essential core, removing unnecessary details.",Brevity
...,...
Rephrase the text as a collection of motivational quotes.,Summarization
Identify real-world companies or organizations relevant to the text's topic.,Summarization
Repurpose the text as instructions for a DIY craft or project.,Summarization
